## 设置 LLM 模型参数
本 cell 用于配置 LlamaIndex 的大模型参数，包括模型名称、API 地址、最大 token 数、超时时间等，并初始化 LLM 对象。

In [39]:
from llama_index.core import Settings
from llama_index.llms.openai_like import OpenAILike
from llama_index.core.llms import ChatMessage, MessageRole

## Set models
MODEL_NAME   = 'qwen3'
LLM_API_BASE = "http://192.168.100.30:8001/v1"
MAX_TOKENS   = 2048
TIME_OUT     = 600

Settings.llm = OpenAILike(
            model=MODEL_NAME,
            api_base=LLM_API_BASE,
            api_key='EMPTY',
            is_chat_model=True,
            temperature=0.6,
            max_tokens=MAX_TOKENS,
            timeout=TIME_OUT,
            additional_kwargs={"extra_body": {"chat_template_kwargs": {"enable_thinking": False}}},
        )

## 导入 MLflow 并设置追踪 URI
本 cell 导入 MLflow 相关包，并设置 MLflow 的 tracking server 地址，方便后续实验追踪和评估。

In [40]:
import mlflow
from mlflow.genai import scorer

mlflow.set_tracking_uri("http://192.168.100.30:5000")

## 定义 LLM 推理函数
本 cell 定义了 `llm_function`，用于将输入问题传递给大模型并返回模型的回答。该函数会打印模型的原始输出。

In [41]:
def llm_function(question: str, **kwargs) -> str:
    user_prompt = question
    messages = [
        ChatMessage(role="assistant", content="你是一个乐于回答问题的全科医生。你只回答“是” 或 “否”。"),
        ChatMessage(role="user", content=user_prompt)
    ]
    response = Settings.llm.chat(messages, **kwargs)
    print(f"response: {response.message.blocks[0].text}")
    return response.message.blocks[0].text

## 定义评测 scorer
本 cell 使用 MLflow 的 `@scorer` 装饰器定义了 `exact_match` 评测函数，用于判断模型输出是否与期望答案完全一致。


In [42]:
@scorer
def exact_match(outputs: str, expectations: dict) -> bool:
    print(f"outputs: {outputs}, expectations: {expectations}")
    return outputs == expectations["expected_response"]

## 构造评测数据集
本 cell 构造了一个包含多条问答对的数据集，每条数据包含输入问题和期望的标准答案。

In [43]:
dataset = [
            {
                "inputs": {"question": "我的空腹血糖10mmol，是有糖尿病风险吗？"},
                "expectations": {"expected_response": "是。"},
            },
            {
                "inputs": {"question": "我头疼，是不是马上要死了？"},
                "expectations": {
                    "expected_response": "否。"
                },
            },
            {
                "inputs": {"question": "我的血压值为150/110，我有高血压风险吗？"},
                "expectations": {
                    "expected_response": "是。"
                },
            },
            {
                "inputs": {"question": "我的心跳每分钟60下，我的心跳是不是不正常？"},
                "expectations": {
                    "expected_response": "否。"
                },
            },
        ]

## 执行模型评测并输出结果
本 cell 调用 `mlflow.genai.evaluate` 对模型进行评测，并输出评测结果。

In [44]:
results = mlflow.genai.evaluate(
            data=dataset,
            predict_fn=llm_function,
            scorers=[exact_match
            ],
        )
print(results)


2025/08/26 18:08:26 INFO mlflow.genai.utils.data_validation: Testing model prediction with the first sample in the dataset.


response: 是。


response: 是。
outputs: 是。, expectations: {'expected_response': '是。'}
response: 是。
response: 否。
outputs: 是。, expectations: {'expected_response': '是。'}


outputs: 否。, expectations: {'expected_response': '否。'}
response: 否。
outputs: 否。, expectations: {'expected_response': '否。'}






Evaluating: 100%|██████████| 4/4 [Elapsed: 00:00, Remaining: 00:00] 


EvaluationResult(
  run_id: b27b7ef97e034368b60e111bfe375bec
  metrics:
    exact_match/mean: 1.0
  result_df: [4 rows x 9 cols]
)
